# Tutorial 1: ESG Report Analysis

**By Tobias Schimanski and based on my paper: "Bridiging the Gap in ESG Measurement: Using NLP to Quantify Environmental, Social, and Governance Communication"**

This notebook serves as a basis for applying AI Natural Language Processing tools to corporate sustainability reports. It is particularily tailored for newcomers to the field with limited or no prior experience. In this tutorial, you can only press play and will be fine.

If you are already advanced, you might just [want to read the paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4622514) and or go directly to the [models on HuggingFace](https://huggingface.co/ESGBERT).

**In this tutorial, we learn to analyse a annual report towards ESG risks and opportunities.**

## Import libraries

We start by importing some standard libraries as well as the "transformers" one. This will help us to use the tools.

In [ ]:
!pip install transformers
!pip install tika

In [ ]:
# standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline # for using the models

import spacy # for sentence extraction
from tika import parser # for the report extraction

## Step 1: Get a feeling for the models

We are going to use models of the [ESG BERT project](https://huggingface.co/ESGBERT). In this tutorial, we don't use GPUs which would speed up the process enourmously.

In [ ]:
### Load the models (takes ca. 1 min)
# Environmental model.
name = "ESGBERT/EnvironmentalBERT-environmental" # path to download from HuggingFace
# In simple words, the tokenizer prepares the text for the model and the model classifies the text-
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSequenceClassification.from_pretrained(name)
# The pipeline combines tokenizer and model to one process.
pipe_env = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Also load the social and governance model.
# Social model.
name = "ESGBERT/SocialBERT-social"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSequenceClassification.from_pretrained(name)
pipe_soc = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Governance model.
name = "ESGBERT/GovernanceBERT-governance"
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSequenceClassification.from_pretrained(name)
pipe_gov = pipeline("text-classification", model=model, tokenizer=tokenizer)

Now, we downloaded models, tokenizer and create the pipeline. This means, we're already ready to go with the classification.

In [ ]:
# You can input single sentences or arrays of sentences into the pipeline,
sentences_test = ["Besides financial considerations, we also consider harms to the biodiversity and broader ecosystem impacts.",
                  "Scope 1 emissions are reported here on a like-for-like basis against the 2013 baseline and exclude emissions from additional vehicles used during repairs.",
                  "Tokenization is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning."]
test = pipe_env(sentences_test)
print(test)

In [ ]:
# Only output the label of the first sentence.
print(test[0]["label"]) # label of first element

You can observe an array of lists as output. The first value gives you the label which could be "environmental" or "none". Furthermore, the 'score' gives you an indication of the confidence of the model in its decision. Play around yourself with the other models and produce example outputs.

In [ ]:
# Try it yourself. Insert a sentence.
sentences_test_2 = ["A"]
test = pipe_soc(sentences_test_2)
print(test)

## Step 2: Extract texts from PDFs

Since this tutorial aims to classify the texts in a complete report, we now download an annual report and split it into single sentences.

In [2]:
# We use the AUDI annual report to analyse in this example.
#path = "https://www.audi.com/content/dam/gbp2/downloads/report/annual-reports/2022/en/audi-report-2022.pdf"
path = "/Users/andrew.mueller/Library/CloudStorage/OneDrive-UniversityofDenver/General - ESG Collaboration-g/data/ESG_reports_update_2023_JS/ESG/AHH_2023.pdf"
# The from_file() function of tika helps us to load the content of the document. (take ca. 30 sec)
report = parser.from_file(path)

NameError: name 'parser' is not defined

In [ ]:
# Have a look at the raw content extracted from the PDF.
print(report["content"])

Now, we want to bring this content on sentence-level to classify it.

In [ ]:
# For this, we use the nlp() function from spacy. (takes 20 secs)
nlp = spacy.load('en_core_web_sm')
about_doc = nlp(report["content"])

This allows us to have every text sequence of the report. We can look at the results. This may look a look a bit complicated, but we just transfer the output to a list of text sequences.

In [ ]:
# We transfer the sequences ("about_doc.sents") to a list of raw strings.
sequences = list(map(str, about_doc.sents))
# Look at the first 10 text sequences.
sequences[:10]

Looking at the output, it becomes apparent that we have many sequences that are actually no sentences, e.g. headlines. Thus, in this prototype, we use a row of naive filters to improve the output quality.

In [ ]:
# "\n" signals a new line. We remove this so that the output looks better.
sentences = [x.replace("\n", "") for x in sequences]

# Remove all empty text, i.e. if the value is "", i.e are empty.
sentences = [x for x in sentences if x != ""]

# A sentence should start with upper case.
sentences = [x for x in sentences if x[0].isupper()]

sentences[:10]

Much more sophisticated filters are possible but also beyond this tutorial.

## Step 3: Classify the whole report

Now, we want to classify the texts. In this demo, we will only classify a subset of the texts towards their environmental label because we are not yet using GPUs (see next tutorials) which will speed up the process immensely. This reduces the time for classification from minutes to seconds. [Full classification now takes around 5 minutes.]

In [ ]:
# Classify subset of sentences.
# The padding and trunctuation parameters help us with classifying texts of different length.
sub_sentences = sentences[:100] # takes around 20 seconds
env = pipe_env(sub_sentences, padding=True, truncation=True)

# You might only want the labels.
env_labels = [x["label"] for x in env]

In [ ]:
# Let's look at the results. We use a dataframe for this purpose.
data_env = pd.DataFrame({"sentence": sub_sentences, "environmental": env_labels})
# Which sentences are labeled as environmental?
data_env[data_env["environmental"] == "environmental"]
# Click the first blue symbol that looks like a calculator in the top right of the dataframe to have a closer look.
# Click the first blue symbol in the top right of the dataframe to have a closer look.

In [ ]:
# It could also be interesting to look at the proporation of environmental sentences.
print(data_env.groupby("environmental").count())
data_env.groupby("environmental").count().plot(kind="bar")

This procedure is easily scalable to whole reports or a large number of reports.

## Step 4: Use advanced language patterns

Besides the sole environmental classification, the real value may lie in advanced language patterns. One question of interest might be if the company perceives climate as a risk, neutral or as an opportunity. We borrow models from a related project to identify this: ClimateBERT.

In [ ]:
# To load the model, we use the exact same steps as above.
model_name = "climatebert/distilroberta-base-climate-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)
pipe_sent = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
# Use the model on the dataset.
sentiment = pipe_sent(sub_sentences, padding=True, truncation=True)
# Add the sentiment to the DataFrame.
data_env["sentiment"] = [x["label"] for x in sentiment]

In [ ]:
# Again, have a look at the outcome.
data_env[data_env["environmental"] == "environmental"]
# Click the first blue symbol that looks like a calculator in the top right of the dataframe to have a closer look.
# Audi seems to display a enthusiastic, pironeering spirit towards sustainability. This positive attitude is reflected as "opportunity".

In [ ]:
# Let's plot the distribution again.
print(data_env[data_env["environmental"] == "environmental"].groupby("sentiment").count()["environmental"])
data_env[data_env["environmental"] == "environmental"].groupby("sentiment").count()["environmental"].plot(kind = "bar", figsize=(5, 3))

With that, we close today's tutorial. I hope, you found this helpful. I will update this file once the new tutorial on GPUs and large-scale analysis is out. Until then, feel free to read the [paper on the ESG BERTs](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4622514) or follow [me on LinkedIn](https://www.linkedin.com/in/tobias-schimanski-964077182/) to not miss out on new developments.